# OSM API

https://wiki.openstreetmap.org/wiki/Osmapi

https://github.com/metaodi/osmapi

In [1]:
import osmapi as osm
from PIL import Image
import requests
from io import BytesIO

https://www.openstreetmap.org/query?lat=47.3777&lon=8.5396

In [ ]:
api = osm.OsmApi() # this instantiate the OsmApi class - do you need to authorise?
node = api.NodeGet('...')

In [3]:
# Can you get also ways and relations?

In [4]:
# Visualization of attributes
response = requests.get(node['tag']['image'])
img = Image.open(BytesIO(response.content))

NameError: name 'node' is not defined

# OSMPythonTools

https://github.com/mocnik-science/osm-python-tools

## Example 1

_Which object does the way with the id 5887599 represent?_

In [5]:
from OSMPythonTools.api import Api
api = Api()
way = api.query('way/5887599')

[api] downloading data: way/5887599


In [6]:
way.tag('building')
way.tag('architect')
way.tag('website')

'https://www.belvedere.at/'

## Example 2

_What is the English name of the church called ‘Stephansdom’, what address does it have, and which of which denomination is the church?_

In [7]:
from OSMPythonTools.overpass import Overpass
overpass = Overpass()
result = overpass.query('way["name"="Stephansdom"]; out body;')

[overpass] downloading data: [timeout:25][out:json];way["name"="Stephansdom"]; out body;


In [8]:
stephansdom = result.elements()[0]

In [9]:
stephansdom.tag('name:en')
'%s %s, %s %s' % (stephansdom.tag('addr:street'), stephansdom.tag('addr:housenumber'), stephansdom.tag('addr:postcode'), stephansdom.tag('addr:city'))
stephansdom.tag('building')
stephansdom.tag('denomination')

'roman_catholic'

## Example 3

_How many trees are in the OSM data of Vienna? And how many trees have there been in 2013?_

In [10]:
from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()
areaId = nominatim.query('Vienna, Austria').areaId()

[nominatim] downloading data: search


In [11]:
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
overpass = Overpass()
query = overpassQueryBuilder(area=areaId, elementType='node', selector='"natural"="tree"', out='count')
result = overpass.query(query)
result.countElements()

[overpass] downloading data: [timeout:25][out:json];area(3600109166)->.searchArea;(node["natural"="tree"](area.searchArea);); out count;


141750

In [12]:
result = overpass.query(query, date='2013-01-01T00:00:00Z', timeout=60)
result.countElements()

[overpass] downloading data: [timeout:60][date:"2013-01-01T00:00:00Z"][out:json];area(3600109166)->.searchArea;(node["natural"="tree"](area.searchArea);); out count;


127690

## Example 4

_Where are waterbodies located in Vienna?_

In [13]:
from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()
areaId = nominatim.query('Vienna, Austria').areaId()

In [14]:
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
overpass = Overpass()
query = overpassQueryBuilder(area=areaId, elementType=['way', 'relation'], selector='"natural"="water"', includeGeometry=True)
result = overpass.query(query)

[overpass] downloading data: [timeout:25][date:"2013-01-01T00:00:00Z"][out:json];area(3600109166)->.searchArea;(way["natural"="water"](area.searchArea);relation["natural"="water"](area.searchArea);); out body geom;
Exception: [overpass] runtime error: Query timed out in "query" at line 1 after 41 seconds.
NoneType: None
[overpass] error in result (cache/overpass-9dc297c5a20c19a14697771e44109695566d12f3): [timeout:25][date:"2013-01-01T00:00:00Z"][out:json];area(3600109166)->.searchArea;(way["natural"="water"](area.searchArea);relation["natural"="water"](area.searchArea);); out body geom;
NoneType: None


Exception: [overpass] error in result (cache/overpass-9dc297c5a20c19a14697771e44109695566d12f3): [timeout:25][date:"2013-01-01T00:00:00Z"][out:json];area(3600109166)->.searchArea;(way["natural"="water"](area.searchArea);relation["natural"="water"](area.searchArea);); out body geom;

In [ ]:
firstElement = result.elements()[0]
firstElement.geometry()

## Example 5

_How did the number of trees in Berlin, Paris, and Vienna change over time?_

In [ ]:
from collections import OrderedDict
from OSMPythonTools.data import Data, dictRangeYears, ALL
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

In [ ]:
dimensions = OrderedDict([
    ('year', dictRangeYears(2013, 2017.5, 1)),
    ('city', OrderedDict({
        'berlin': 'Berlin, Germany',
        'paris': 'Paris, France',
        'vienna': 'Vienna, Austria',
    })),
])

In [ ]:
overpass = Overpass()
def fetch(year, city):
    areaId = nominatim.query(city).areaId()
    query = overpassQueryBuilder(area=areaId, elementType='node', selector='"natural"="tree"', out='count')
    return overpass.query(query, date=year, timeout=120).countElements()
data = Data(fetch, dimensions)

In [ ]:
data.plot(city=ALL, filename='example4.png')

In [ ]:
data.select(city=ALL).getCSV()

# One more example with Overpass_API

https://wiki.openstreetmap.org/wiki/Overpass_API

https://medium.com/better-programming/how-to-get-open-street-and-map-data-using-python-2b777bf5af14

`node["amenity"="cafe"]({{bbox}}); out;`

`node["amenity"="pub"](53.2987342,-6.3870259,53.4105416,-6.1148829); out;
`

`( node["amenity"="cafe"]({{bbox}});
  way["amenity"="cafe"]({{bbox}});
  relation["amenity"="cafe"]({{bbox}});
);
out;`

`area["ISO3166-1"="GB"][admin_level=2];
node["place"="city"](area);
out;`

In this case, area IDs need to be derived from an existing OSM way by adding `2400000000` to its OSM ID, or, in case of relation, by adding `3600000000`

`area(3600062149);
node["place"="city"](area);
out;`

The first set of values can control the verbosity or the detail of information of the output, such as `ids`, `skel`, `body`(default value), `tags`, `meta`, and `count`

For more information please follow the link: https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL

In [ ]:
import requests
import json
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="DE"][admin_level=2];
(node["amenity"="biergarten"](area);
 way["amenity"="biergarten"](area);
 rel["amenity"="biergarten"](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Collect coords into list
coords = []
for element in data['elements']:
    if element['type'] == 'node':
        lon = element['lon']
        lat = element['lat']
        coords.append((lon, lat))
    elif 'center' in element:
        lon = element['center']['lon']
        lat = element['center']['lat']
        coords.append((lon, lat))
# Convert coordinates into numpy array
X = np.array(coords)
plt.plot(X[:, 0], X[:, 1], 'o')
plt.title('Biergarten in Germany')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')
plt.show()

In [ ]:
import overpy
api = overpy.Overpass()
r = api.query("""
area["ISO3166-1"="DE"][admin_level=2];
(node["amenity"="biergarten"](area);
 way["amenity"="biergarten"](area);
 rel["amenity"="biergarten"](area);
);
out center;
""")
coords  = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

In [ ]:
# Convert coordinates into numpy array
X = np.array(coords)
plt.plot(X[:, 0], X[:, 1], 'o')
plt.title('Biergarten in Germany')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')
plt.show()